# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bredasdorp,65,ZA,1669846783,81,-34.5322,20.0403,63.12,2.44
1,Kruisfontein,8,ZA,1669846573,72,-34.0033,24.7314,64.89,4.54
2,Port Hardy,66,CA,1669846518,75,50.6996,-127.4199,27.99,14.38
3,Port Alfred,100,ZA,1669846559,94,-33.5906,26.8910,66.18,1.01
4,Båtsfjord,75,NO,1669846784,86,70.6345,29.7185,31.78,9.22
...,...,...,...,...,...,...,...,...,...
559,Sur,88,OM,1669846928,56,22.5667,59.5289,73.63,1.97
560,Long Xuyen,100,VN,1669846929,93,10.3833,105.4167,76.95,3.51
561,Rome,100,US,1669846643,88,43.2128,-75.4557,41.38,12.66
562,Yumen,66,CN,1669846929,78,40.2833,97.2000,4.89,2.64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [7]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
41,Eufaula,0,US,1669846793,40,31.8913,-85.1455,71.64,4.61
139,Manyana,0,BW,1669846819,29,-23.4000,21.7167,76.26,7.67
150,Arlit,0,NE,1669846764,20,18.7369,7.3853,70.47,6.78
255,Luangwa,0,ZM,1669846848,77,-15.6167,30.4167,72.39,3.27
256,Zabid,0,YE,1669846849,74,14.1951,43.3152,76.33,7.61
317,Cayenne,0,GF,1669846659,83,4.9333,-52.3333,78.84,2.30
320,Vallenar,0,CL,1669846864,54,-28.5708,-70.7581,72.10,7.96
493,Dwārka,0,IN,1669846910,55,22.2394,68.9678,74.71,9.53
524,Alice Springs,0,AU,1669846919,26,-23.7000,133.8833,71.15,0.00
552,Bosaso,0,SO,1669846927,78,11.2842,49.1816,76.39,1.52


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
41,Eufaula,US,31.8913,-85.1455,
139,Manyana,BW,-23.4000,21.7167,
150,Arlit,NE,18.7369,7.3853,
255,Luangwa,ZM,-15.6167,30.4167,
256,Zabid,YE,14.1951,43.3152,
317,Cayenne,GF,4.9333,-52.3333,
320,Vallenar,CL,-28.5708,-70.7581,
493,Dwārka,IN,22.2394,68.9678,
524,Alice Springs,AU,-23.7000,133.8833,
552,Bosaso,SO,11.2842,49.1816,


In [26]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [27]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    time.sleep(1)

print("-------End of Search-------")

Retrieving Results for Index 41: Eufaula.
Missing field/result... skipping.
------------
Retrieving Results for Index 139: Manyana.
Missing field/result... skipping.
------------
Retrieving Results for Index 150: Arlit.
Missing field/result... skipping.
------------


KeyboardInterrupt: 

In [9]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
135,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
207,Necochea,AR,-38.55,-58.74,Hotel Real
263,Balabac,PH,7.99,117.06,Sing and Swing Lodge
357,Faxinal,BR,-24.00,-51.32,Hotel Shalom
373,San Vicente,AR,-26.62,-54.13,Hospedaje La Luz
398,Cockburn Town,TC,21.46,-71.14,Bohio Dive Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))